In [ ]:
import pymysql#db
import sys
from future.utils import iteritems
from collections import Counter#갯수
import os

from math import sqrt
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc #한글이 나오게
import csv
import re
import operator

##db
db = pymysql.connect(host='localhost', user='root', password='****' ,db='documents', charset='utf8')
# prepare a cursor object using cursor() method


cursor = db.cursor()

# execute SQL query using execute() method.

cursor.execute("SELECT VERSION()")
# Fetch a single row using fetchone() method.

data = cursor.fetchone()

#db연결확인
print("Database version : %s " % data)


#csv형식의 뉴스 테이블(culture) 읽기, 읽은 후 tags 리스트에 태그들 저장
tags=[]
user={}

try:
    with db.cursor() as cursor:
        sql = "select tags1,tags2,tags3,tags4,tags5,tags6 from culture"
        cursor.execute("set names utf8")
        cursor.execute(sql)
        result = cursor.fetchall()
        
        for i in result: 
            tags.extend(i)
finally:
    db.close()
    
#갯수세기
count = Counter(tags)

#user딕트에 저장
user['bm'] = count
user['kk'] = count
user['dd'] = count

#plt폰트
path = '/Windows/Fonts/malgun.ttf'
font_name = font_manager.FontProperties(fname=path, size=10).get_name()
rc('font', family=font_name)

def drawGraph(data, name1, name2):
    plt.figure(figsize=(14,8)) # plot 크기설정
    
    # plot 좌표를 위한 list 선언
    li = []
    li2 = []
    
    for i in critics[name1]: # i = 키 값
        if i in data[name2]: # 같은 영화를 평가했을때만
            li.append(critics[name1][i]) # name1의 평점 li[]에 추가
            li2.append(critics[name2][i]) # name2의 평점 li2[]에 추가
            plt.text(critics[name1][i],critics[name2][i],i) # 영화 제목 text 찍기
            
    plt.plot(li,li2,'ro') #plot그리기
    
    #각 축의 크기 설정 (0에서 6까지)
    plt.axis([0,6,0,6])
 
    # x축과 y축 이름 설정
    plt.xlabel(name1)
    plt.ylabel(name2)
 
    # 그리기
    plt.show()
#drawGraph(critics,'황해도','임명진')
#drawGraph(critics,'심수정','차현석')

    # 피어슨 상관계수 구하기
def sim_pearson(data, name1, name2):
    sumX=0 # X의 합
    sumY=0 # Y의 합
    sumPowX=0 # X 제곱의 합
    sumPowY=0 # Y 제곱의 합
    sumXY=0 # X*Y의 합
    count=0 #영화 개수
    
    for i in data[name1]: # i = key name1 비교대상 name2 비교대상빼고 나머지       
        if i in data[name2]: # 같은 태그를 평가했을때만 
            count+=1
            sumX+=data[name1][i]
            sumY+=data[name2][i]
            sumPowX+=pow(data[name1][i],2)
            sumPowY+=pow(data[name2][i],2)
            sumXY+=data[name1][i]*data[name2][i]
            
    if (count<2):#태그 최소 같은게 2개이상일때만
        return 0
    else:
        return ( sumXY- ((sumX*sumY)/count) )/ sqrt( (sumPowX - (pow(sumX,2) / count)) * (sumPowY - (pow(sumY,2)/count)))
    
    
#sim_pearson(critics,'황해도','임명진')
#sim_pearson(critics, '차현석','박병관')

# 딕셔너리 돌면서 상관계수순으로 정렬 index=인원수
def top_match(data, name, index=1, sim_function=sim_pearson):
    li={}
    li_dict={}
    
    for i in data: #딕셔너리를 돌고
        if name!=i: #자기 자신이 아닐때만
             li_dict[i] = sim_function(data,name,i)
            #li.append((sim_function(data,name,i),i)) #sim_function()을 통해 상관계수를 구하고 li[]에 추가
    #li.reverse() #내림차순
    #return li[:index]
    return li_dict
                          
def show(data):
    
    print(data.values())
    sorted_data=sorted(data.items(), key=lambda x: x[1], reverse=True)#내림차순정렬
    max_name = max(data.items(), key=operator.itemgetter(1))[0]
    
    print(sorted_data)
    print(max_name)
    
show(top_match(user, 'bm'))

#score= []
#names= []
#for i in top_match(critics, '차현석',7):
#    score.append(i[0])
#    names.append(i[1])

#그래프로나타내기

#def barchart(data, labels): # data, labels는 list형태로 사용
#    positions = range(len(data))
#    plt.barh(positions, data, height=0.5,color='r') #가로
#    plt.yticks(positions, labels)
#    plt.xlabel('similarity',fontproperties=font_name) #x축
#    plt.ylabel('제목',fontproperties=font_name) #y축
#    plt.show() #출력
    
#barchart(score,names)